In [0]:
df = spark.table('workspace.bronze.crm_prd_info')
display(df)

In [0]:
%sql
SELECT prd_id, prd_key, prd_nm, prd_cost, prd_line, prd_start_dt, prd_end_dt, COUNT(*) AS duplicate_count
FROM workspace.bronze.crm_prd_info
GROUP BY prd_id, prd_key, prd_nm, prd_cost, prd_line, prd_start_dt, prd_end_dt
HAVING COUNT(*) > 1

In [0]:
%sql
SELECT prd_id, prd_key, prd_nm, prd_cost, prd_line, prd_start_dt, prd_end_dt,
       CASE WHEN prd_key != TRIM(prd_key) OR LENGTH(prd_key) != LENGTH(TRIM(prd_key)) THEN 'extra_spaces' ELSE '' END AS prd_key_flag,
       CASE WHEN prd_nm != TRIM(prd_nm) OR LENGTH(prd_nm) != LENGTH(TRIM(prd_nm)) THEN 'extra_spaces' ELSE '' END AS prd_nm_flag,
       CASE WHEN prd_line != TRIM(prd_line) OR LENGTH(prd_line) != LENGTH(TRIM(prd_line)) THEN 'extra_spaces' ELSE '' END AS prd_line_flag
FROM workspace.bronze.crm_prd_info
WHERE (prd_key != TRIM(prd_key) OR LENGTH(prd_key) != LENGTH(TRIM(prd_key)))
   OR (prd_nm != TRIM(prd_nm) OR LENGTH(prd_nm) != LENGTH(TRIM(prd_nm)))
   OR (prd_line != TRIM(prd_line) OR LENGTH(prd_line) != LENGTH(TRIM(prd_line)))

In [0]:
%sql
-- Check for missing values in prd_start_dt and prd_end_dt
SELECT COUNT(*) AS missing_start_count
FROM workspace.bronze.crm_prd_info
WHERE prd_start_dt IS NULL;

SELECT COUNT(*) AS missing_end_count
FROM workspace.bronze.crm_prd_info
WHERE prd_end_dt IS NULL;

-- Fix non-date values in prd_start_dt and prd_end_dt by attempting to parse them to DATE
UPDATE workspace.bronze.crm_prd_info
SET prd_start_dt = TRY_CAST(prd_start_dt AS DATE)
WHERE CAST(prd_start_dt AS STRING) NOT RLIKE '^[0-9]{4}-[0-9]{2}-[0-9]{2}$';

UPDATE workspace.bronze.crm_prd_info
SET prd_end_dt = TRY_CAST(prd_end_dt AS DATE)
WHERE CAST(prd_end_dt AS STRING) NOT RLIKE '^[0-9]{4}-[0-9]{2}-[0-9]{2}$';

-- Show sample date values and their format
SELECT DISTINCT prd_start_dt
FROM workspace.bronze.crm_prd_info
ORDER BY prd_start_dt DESC;

SELECT DISTINCT prd_end_dt
FROM workspace.bronze.crm_prd_info
ORDER BY prd_end_dt DESC;

In [0]:
%sql
-- Check for missing values in prd_id and prd_cost
SELECT COUNT(*) AS missing_prd_id_count
FROM workspace.bronze.crm_prd_info
WHERE prd_id IS NULL;

SELECT COUNT(*) AS missing_prd_cost_count
FROM workspace.bronze.crm_prd_info
WHERE prd_cost IS NULL;

-- Check for non-numeric values (should be empty if column is type INT)
SELECT prd_id
FROM workspace.bronze.crm_prd_info
WHERE CAST(prd_id AS STRING) RLIKE '[^0-9]';

SELECT prd_cost
FROM workspace.bronze.crm_prd_info
WHERE CAST(prd_cost AS STRING) RLIKE '[^0-9]';

-- Check for outliers in prd_id and prd_cost (example: values outside expected range)
SELECT prd_id
FROM workspace.bronze.crm_prd_info
WHERE prd_id < 0 OR prd_id > 99999999;

SELECT prd_cost
FROM workspace.bronze.crm_prd_info
WHERE prd_cost < 0 OR prd_cost > 1000000;

In [0]:
# List unique values and counts for product line
from pyspark.sql import functions as F

line_counts = spark.table('workspace.bronze.crm_prd_info') \
    .groupBy('prd_line') \
    .count() \
    .orderBy('count', ascending=False)
display(line_counts)

# Flag unexpected values (not 'R', 'S', 'M', null)
expected = {'R', 'S', 'M', None}
unique_lines = [row['prd_line'] for row in line_counts.collect()]
issues = [val for val in unique_lines if val not in expected]
print("Unexpected or ambiguous product line values:", issues)

In [0]:
%sql
-- Standardize prd_key for joinability
SELECT prd_id,
       UPPER(TRIM(REGEXP_REPLACE(prd_key, '[^A-Za-z0-9]', ''))) AS std_prd_key,
       prd_nm,
       prd_cost,
       prd_line,
       prd_start_dt,
       prd_end_dt
FROM workspace.bronze.crm_prd_info

In [0]:
from pyspark.sql import functions as F

def trim_all_string_columns(df):
    for col_name, dtype in df.dtypes:
        if dtype == 'string':
            df = df.withColumn(col_name, F.trim(F.col(col_name)))
    return df

# Read trimmed DataFrame from previous cell
trimmed_df = spark.table('workspace.bronze.crm_prd_info')
trimmed_df = trim_all_string_columns(trimmed_df)

# Add standardized product key column
silver_df = trimmed_df.withColumnRenamed('prd_id', 'product_id') \
    .withColumnRenamed('prd_key', 'product_key') \
    .withColumnRenamed('prd_nm', 'product_name') \
    .withColumnRenamed('prd_cost', 'product_cost') \
    .withColumnRenamed('prd_line', 'product_line') \
    .withColumnRenamed('prd_start_dt', 'start_date') \
    .withColumnRenamed('prd_end_dt', 'end_date')

# Insert std_prd_key right after product_key, matching SQL logic
cols = silver_df.columns
idx = cols.index('product_key') + 1
cols_new = cols[:idx] + ['std_prd_key'] + cols[idx:]
silver_df = silver_df.withColumn(
    'std_prd_key',
    F.upper(F.trim(F.regexp_replace(F.col('product_key'), '[^A-Za-z0-9]', '')))
).select(cols_new)

# Write to silver table
silver_df.write.mode('overwrite').saveAsTable('workspace.silver.crm_prd_info')
display(silver_df)